# Webscraper
#### TX Death Row Last Words
#### Kwadwo Alfajiri Shah
#### 9 July 2024

Webscraping contains two main tasks
* Scraping the main table of [TX Death Row Information](https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html), which includes identifying information for prisoners and links to personal details and last words.
* Programmatically accessing the links in the main table and scraping prisoner details and last words from all.

In [338]:
import requests
import bs4
import pandas as pd

from datetime import date

### 1. Scraping the main table

#### Access webpage

In [216]:
## review
url = 'https://www.tdcj.texas.gov/death_row/dr_executed_offenders.html'
headers = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

In [217]:
## access
status = requests.get(url, headers = headers, verify = False) # verify = True results in an SSL error. not the best practice to proceed without verifying TLS, but...
status

C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [218]:
status.close()

#### Parse results

In [219]:
## parse
soup = bs4.BeautifulSoup(status.text, features = 'html.parser')
html_table = soup.find_all('tr')

#### Transform

Get column names:

In [220]:
columns_html = html_table.pop(0)
columns_html

<tr>
<th scope="col" style="text-align: center">Execution</th>
<th scope="col" style="text-align: center; width: 16%">Link</th>
<th scope="col" style="text-align: center; width: 13%">Link</th>
<th scope="col" style="text-align: center">Last Name</th>
<th scope="col" style="text-align: center">First Name</th>
<th scope="col" style="text-align: center; width: 7%">TDCJ<br/>Number</th>
<th scope="col" style="text-align: center">Age</th>
<th scope="col" style="text-align: center">Date</th>
<th scope="col" style="text-align: center">Race</th>
<th scope="col" style="text-align: center">County</th>
</tr>

In [222]:
## create list of col names
columns = []

for tag in columns_html.find_all('th'):
    column = tag.text
    columns.append(column)

# custom names (webpage names are both simply 'Link')
columns[1] = "Inmate Information"
columns[2] = "Last Words"


columns

['Execution',
 'Inmate Information',
 'Last Words',
 'Last Name',
 'First Name',
 'TDCJNumber',
 'Age',
 'Date',
 'Race',
 'County']

Create data table:

In [223]:
## create list of table values
executed_list = []

for row in html_table:
    row_values = []
    for cell in row.find_all('td'):
        if cell.a is None:
            cell_value = cell.text
        else:
            cell_value = cell.a['href']
        row_values.append(cell_value)
    executed_list.append(row_values)

In [224]:
## convert list to df
executed = pd.DataFrame(executed_list, columns = columns)
executed.head(10)

,Execution,Inmate Information,Last Words,Last Name,First Name,TDCJNumber,Age,Date,Race,County
0,588,dr_info/gonzalesramiro.html,dr_info/gonzalesramirolast.html,Gonzales,Ramiro,999513,41,6/26/2024,Hispanic,Medina
1,587,dr_info/cantuivan.html,dr_info/cantuivanlast.html,Cantu,Ivan,999399,50,2/28/2024,Hispanic,Collin
2,586,dr_info/renteriadavid.html,dr_info/renteriadavidlast.html,Renteria,David,999460,53,11/16/2023,Other,El Paso
3,585,dr_info/brewer.jpg,dr_info/brewerbrentlast.html,Brewer,Brent,999000,53,11/9/2023,White,Randall
4,584,dr_info/murphyjedidiah.html,dr_info/murphyjedidiahlast.html,Murphy,Jedidiah,999392,48,10/10/2023,White,Dallas
5,583,dr_info/brownarthur.jpg,dr_info/brownarthurlast.html,"Brown, Jr.",Arthur,999110,52,3/9/2023,Black,Harris
6,582,dr_info/greengary.html,dr_info/greengarylast.html,Green,Gary,999561,51,3/7/2023,Black,Dallas
7,581,dr_info/balentinejohn.html,dr_info/balentinejohnlast.html,Balentine,John,999315,54,2/9/2023,Black,Potter
8,580,dr_info/ruizwesley.html,dr_info/ruizwesleylast.html,Ruiz,Wesley,999536,43,2/1/2023,Hispanic,Dallas
9,579,dr_info/frattarobert.jpg,dr_info/frattarobertlast.html,Fratta,Robert,999189,65,1/10/2023,White,Harris


In [225]:
## save
executed.to_csv('../data/executed_inmates.csv', index=False)

## 2. Scraping prisoner info and last words

In [260]:
executed = pd.read_csv('../data/executed_inmates.csv')
executed.head()

,Execution,Inmate Information,Last Words,Last Name,First Name,TDCJNumber,Age,Date,Race,County
0,588,dr_info/gonzalesramiro.html,dr_info/gonzalesramirolast.html,Gonzales,Ramiro,999513,41,6/26/2024,Hispanic,Medina
1,587,dr_info/cantuivan.html,dr_info/cantuivanlast.html,Cantu,Ivan,999399,50,2/28/2024,Hispanic,Collin
2,586,dr_info/renteriadavid.html,dr_info/renteriadavidlast.html,Renteria,David,999460,53,11/16/2023,Other,El Paso
3,585,dr_info/brewer.jpg,dr_info/brewerbrentlast.html,Brewer,Brent,999000,53,11/9/2023,White,Randall
4,584,dr_info/murphyjedidiah.html,dr_info/murphyjedidiahlast.html,Murphy,Jedidiah,999392,48,10/10/2023,White,Dallas


#### Scraping prisoner information

In [328]:
url_base = 'https://www.tdcj.texas.gov/death_row/'

In [329]:
def scrape_info(path_name):
    '''Scrape prisoner information programatically.'''
    
    url = url_base + path_name
    
    if path_name.endswith('.html'):
    

        print(f'Scraping {url} ...')
    
        ## access
        status = requests.get(url, headers = headers, verify = False) # verify = True results in an SSL error. not the best practice to proceed without verifying TLS, but...
        status.close()
        
        if status.status_code != 200:
            print(f'Invalid status code: {status.status_code}. Conintuing...')
            pass
            
        ## parse
        soup = bs4.BeautifulSoup(status.text, features = 'html.parser')
        html_table = soup.find_all('tr')
        
        ## transform
        table_dict = {}

        for row in html_table:
                row_content = [x for x in row.text.replace('\xa0','').split('\n') if x]
                # handle missing data
                if len(row_content) < 2:
                    row_content.append(None)
                if row_content[0] is not None:
                    table_dict[row_content[0]] = row_content[1]
        
        return(table_dict)
    
    else:
        print(f'Invalid filetype {url} . Continuing...')
        
        
        
    
    

In [330]:
# scraped_info = [scrape_info(pathname) for pathname in info['Inmate Information']]

# using for loop allows to save results even if there's an error in the process
info_list = []
for pathname in executed['Inmate Information']:
    scraped_info = scrape_info(pathname)
    info_list.append(scraped_info)

Scraping https://www.tdcj.texas.gov/death_row/dr_info/gonzalesramiro.html ...


C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Scraping https://www.tdcj.texas.gov/death_row/dr_info/cantuivan.html ...


C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Scraping https://www.tdcj.texas.gov/death_row/dr_info/renteriadavid.html ...


...

C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/pinkertonjay.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/barneyjeffery.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/basscharles.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/rumbaugh.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/porterhenry.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/miltoncharles.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/delarosajesse.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/morinstephen.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/skillerndoyle.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/dr_info/barefootthomas.jpg . Continuing...
Invalid filetype https://www.tdcj.texas.gov/death_

C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Scraping https://www.tdcj.texas.gov/death_row/dr_info/brookscharlie.html ...


C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [331]:
## remove None's
info_list = list(filter(None, info_list))

## convert list to df
info = pd.DataFrame(info_list)
info.head(11)
info

,Name,TDCJ Number,Date of Birth,Date Received,Age (when Received),Education Level (Highest Grade Completed),Date of Offense,Age (at the time of Offense),County,Race,...,Eye Color,Native County,Native State,Age (when Received),Height,Weight,Native Country,Age,Age (when shoulReceived),Photo not available
0,"Gonzales, Ramiro",999513,11/05/1982,09/27/2006,23,7th Grade,01/15/2001,18,Medina,Hispanic,...,Brown,Frio,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Cantu, Ivan Abner",999399,06/14/1973,11/08/2001,NaN,12,11/04/2000,27,Collin,Hispanic,...,Brown,Dallas,Texas,28,NaN,NaN,NaN,NaN,NaN,NaN
2,"Renteria, David Santiago",999460,11/22/1969,11/20/2003,33,12,11/18/2001,31,El Paso,"Other, (Native American)",...,Brown,El Paso,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Murphy, Jedidiah Isaac",999392,09/01/1975,07/26/2001,NaN,12,10/04/2000,25,Dallas,White,...,Brown,Sutton,Texas,25,NaN,NaN,NaN,NaN,NaN,NaN
4,"Green, Gary",999561,03/14/1971,11/22/2010,39,None,09/22/2009,37,Dallas,Black,...,Brown,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,NaN,887,6/11/1965,10/23/1987,22,12,7/23/1986,21,Clay (change of venue from Montague),White,...,Brown,Brunswick,Georgia,NaN,NaN,NaN,NaN,NaN,NaN,Name
188,"Cantu, Ruben Montoya",804,12/5/66,9/10/85,18,9 years,11/8/84,17,Bexar,Hispanic,...,Brown,Bexar,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
189,"Hernandez, Ramon Pedro",667,03/02/1942,09/30/1980,38,9,06/20/1980,38,El Paso,Hispanic,...,Brown,El Paso,Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN
190,"Autry, James David",670,9/27/1954,10/10/1980,NaN,6,4/20/1980,25,Jefferson,White,...,Brown,Potter,Texas,26,NaN,NaN,NaN,NaN,NaN,NaN


#### Save prisoner information

In [341]:
## date
save_date = format(date.today(), '%Y%m%d')
save_date

## save info
path = f'../data/prisoner_info_{save_date}.csv'
info.to_csv(path, index = False)

## 99. Code testing and validation

#### Scraping prisoner information - specific example (developing `scrape_info()`)

In [296]:
url_base = 'https://www.tdcj.texas.gov/death_row/'
url_test = url_base + executed['Inmate Information'].iloc[-562]
url_test

'https://www.tdcj.texas.gov/death_row/dr_info/swearingenlarry.html'

In [297]:
## access
status_test = requests.get(url_test, headers = headers, verify = False) # verify = True results in an SSL error. not the best practice to proceed without verifying TLS, but...
status_test

C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


<Response [200]>

In [298]:
status_test.close()

In [300]:
## parse
soup_test = bs4.BeautifulSoup(status_test.text, features = 'html.parser')
html_table_test = soup_test.find_all('tr')
html_table_test

[<tr>
 <td rowspan="7" style="vertical-align: top"><img alt="Picture of Inmate" src="swearingenlarry2.jpg"/></td>
 <td class="table_deathrow_bold_align_right" style="vertical-align: top">Name</td>
 <td class="table_deathrow_align_left" style="vertical-align: top">Larry Ray Swearingen</td>
 </tr>,
 <tr>
 <td class="table_deathrow_bold_align_right" style="vertical-align: top">TDCJ Number</td>
 <td class="table_deathrow_align_left" style="vertical-align: top">999361</td>
 </tr>,
 <tr>
 <td class="table_deathrow_bold_align_right" style="vertical-align: top">Date of Birth</td>
 <td class="table_deathrow_align_left" style="vertical-align: top">5/21/1971</td>
 </tr>,
 <tr>
 <td class="table_deathrow_bold_align_right" style="vertical-align: top">Date Received</td>
 <td class="table_deathrow_align_left" style="vertical-align: top">7/12/2000</td>
 </tr>,
 <tr>
 <td class="table_deathrow_bold_align_right" style="vertical-align: top">Age (when    Received)</td>
 <td class="table_deathrow_align_lef

In [317]:
## transform html to dict
table_dict = {}

for row in html_table_test:
        row_content = [x for x in row.text.replace('\xa0','').split('\n') if x]
        if len(row_content) < 2:
            row_content.append(None)
        print(row_content)
        if row_content[0] is not None:
            table_dict[row_content[0]] = row_content[1]
        
table_dict

['Name', 'Larry Ray Swearingen']
['TDCJ Number', '999361']
['Date of Birth', '5/21/1971']
['Date Received', '7/12/2000']
['Age (when    Received)', '29']
['Education Level (Highest Grade Completed)', '11']
['Date of Offense', '12/8/1998']
['Age (at the time of Offense)', '27']
['County', 'Montgomery']
['Race', 'White']
['Gender', 'Male']
['Hair Color', 'Brown']
['Height (in Feet and Inches)', '5′ 10″']
['Weight (in Pounds)', '194']
['Eye Color', 'Blue']
['Native County', 'Montgomery']
['Native State', 'Texas']
[None]


{'Name': 'Larry Ray Swearingen',
 'TDCJ Number': '999361',
 'Date of Birth': '5/21/1971',
 'Date Received': '7/12/2000',
 'Age (when    Received)': '29',
 'Education Level (Highest Grade Completed)': '11',
 'Date of Offense': '12/8/1998',
 'Age (at the time of Offense)': '27',
 'County': 'Montgomery',
 'Race': 'White',
 'Gender': 'Male',
 'Hair Color': 'Brown',
 'Height (in Feet and Inches)': '5′ 10″',
 'Weight (in Pounds)': '194',
 'Eye Color': 'Blue',
 'Native County': 'Montgomery',
 'Native State': 'Texas'}

In [162]:
table_dict2 = table_dict
table_dict_list = [table_dict, table_dict2]
test = pd.DataFrame(table_dict_list)
test

,Name,TDCJ Number,Date of Birth,Date Received,Age (when Received),Education Level (Highest Grade Completed),Date of Offense,Age (at the time of Offense),County,Race,Gender,Hair Color,Height (in Feet and Inches),Weight (in Pounds),Eye Color,Native County,Native State
0,"Gonzales, Ramiro",999513,11/05/1982,09/27/2006,23,7th Grade,01/15/2001,18,Medina,Hispanic,Male,Black,5′ 2″,136,Brown,Frio,Texas
1,"Gonzales, Ramiro",999513,11/05/1982,09/27/2006,23,7th Grade,01/15/2001,18,Medina,Hispanic,Male,Black,5′ 2″,136,Brown,Frio,Texas


In [324]:
## testing invalid responses/filetypes
test_paths = ['leedle.html', 'leedle.jpg', 'lee.html']
test = [scrape_info(pathname) for pathname in test_paths]
test

Scraping https://www.tdcj.texas.gov/death_row/leedle.html ...


C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


Invalid status code: 404. Conintuing...
Invalid filetype https://www.tdcj.texas.gov/death_row/leedle.jpg . Continuing...
Scraping https://www.tdcj.texas.gov/death_row/lee.html ...
Invalid status code: 404. Conintuing...


C:\Users\Alfajiri\anaconda3\Lib\site-packages\urllib3\connectionpool.py:1056: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.tdcj.texas.gov'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


[{}, None, {}]

In [325]:
## dropping None's (Invalid filetype) and empty dicts (Invalid status code)
test2 = list(filter(lambda item: item is not None, test))
test2

[{}, {}]

In [326]:
test3 = list(filter(None, test))
test3

[]